# 인스타 그램 크롤링

In [1]:
!python --version

In [2]:
pwd  # 현재 작업하고 있는 위치 파악, 

'C:\\Users\\USER\\Desktop\\python'

실행시키는 파일 내부에 data(excel 저장 위치), image(크롤링한 그림 정장 위치) 폴더가 있어야 코드가 작동

In [3]:
import pandas as pd  # 판다스 - 데이터프레임(엑셀)
import numpy as np   # 넘파이 - 수학 연산

In [4]:
# 라이브러리 import
# 라이브러리 : 필요한 도구
from selenium import webdriver  # 라이브러리(모듈) 가져오라
from selenium.webdriver import ActionChains as AC
from tqdm import tqdm
from tqdm import tqdm_notebook
import re
from time import sleep
import time

In [5]:
# 데이터 수집할 키워드 지정
keywords = ["음식"]
keywords[0]

'음식'

In [6]:
# 크롬창 띄우기
driver = webdriver.Chrome("C:/Users/USER/Desktop/python/chromedriver.exe")  # 크롬 드라이버 로드
driver.get("https://www.instagram.com/explore/tags/%EC%9D%8C%EC%8B%9D/")  # "인스타에서 음식" 검색
time.sleep(5)

크롬에서 인스타를 띄웠을때 로그인을 해야함

In [7]:
# ### html이 제대로 사용되었는지 따로 떼서 확인 해보기 ###

# target_info = {}      

# overlays1 = "._2dDPU.vCf6V .FFVAD"                   # 사진창 속 사진의 html element (이때 띄어쓰기는 . 으로 표현된다, )   
# img = driver.find_element_by_css_selector(overlays1)    # 사진 선택
# pic = img.get_attribute('src')                          # get_attribute는 ()내의 글자가 지칭하고 있는 값을 html에서 불러와서 사용한다. 여기서는 사진의 src 주소를 html에서 불러와서 사용하고 있음.
# target_info['picture'] = pic                            # target_info 라는 딕셔너리에 'picture'라는 이름으로 저장
# print(target_info)


In [8]:
for keyword in keywords:
    dict = {}                                                       # 전체 데이터를 담을 딕셔너리 생성
    # 사진 클릭
    CSS_tran="div.Nnq7C.weEfm"                         # 사진 버튼의 html element ("div.Nnq7C.weEfm")
    tran_button = driver.find_element_by_css_selector(CSS_tran)     # 사진 버튼 찾기
    AC(driver).move_to_element(tran_button).click().perform()       # 사진 버튼 클릭
    time.sleep(1)

    # 크롤링 시작
    len_insta = 20   # 몇 개 글을 크롤링 할건지

    for i in range(0, len_insta):    # range : 숫자가 1씩하는 함수

        target_info = {}                                            # 사진별 데이터를 담을 딕셔너리 생성

        try:    # 크롤링을 시도해라.
            # 사진(pic) 크롤링 시작
            overlays1 = "._2dDPU.vCf6V .FFVAD"                   # 사진창 속 사진의 html element (이때 띄어쓰기는 . 으로 표현된다, )   
            img = driver.find_element_by_css_selector(overlays1)    # 사진 선택
            pic = img.get_attribute('src')                          # get_attribute는 ()내의 글자가 지칭하고 있는 값을 html에서 불러와서 사용한다. 여기서는 사진의 src 주소를 html에서 불러와서 사용하고 있음.
            target_info['picture'] = pic                            # target_info 라는 딕셔너리에 'picture'라는 key 값과 'src 주소'를 value 값으로 저장한다. 
            # print(target_info) 

            # 날짜(date) 크롤링 시작
            overlays2 = "._2dDPU.vCf6V .c-Yi7 > time"                  # 날짜 지정
            datum2 = driver.find_element_by_css_selector(overlays2)     # 날짜 선택
            date = datum2.get_attribute('title')
            target_info['date'] = date
            # print(target_info)

            # 좋아요(like) 크롤링 시작
            overlays3 = ".Nm9Fw"                                        # 좋아요 지정
            datum3 = driver.find_element_by_css_selector(overlays3)     # 좋아요 선택
            like = datum3.text                                          # 좋아요 크롤링 완료
            target_info['like'] = like
            # print(target_info)

            # 해시태그(tag) 크롤링 시작
            overlays4 = ".C7I1f.X7jCj"                                  # 태그 지정
            datum4 = driver.find_element_by_css_selector(overlays4)     # 태그 선택
            tag_raw = datum4.text
            tags = re.findall('#[A-Za-z0-9가-힣]+', tag_raw)            # "javascript에서 사용하는 한글,숫자,영어의 정규표현식으로 #뒤의 문자를 가져오기 위해 사용
            tag = ''.join(tags).replace("#"," ")                        # "#" 제거
            target_info['tag'] = tag
            # print(target_info)

            dict[i] = target_info            # 토탈 딕셔너리로 만들기

            print("{0}".format(i), tag)      # 크롤링한글각각이dict에저장될때마다“몇번째글인지, 해당글의해시태그”를 출력하게 하여 진행상태를체크

            # 다음장 클릭
            hit=0
            while hit < 1:                          # 몇 번에 한번씩 크롤링할 것인지 숫자 지정
                hit+=1
                CSS_tran2="a._65Bje.coreSpriteRightPaginationArrow"             # 다음 버튼 정의
                tran_button2 = driver.find_element_by_css_selector(CSS_tran2)  # 다음 버튼 find
                AC(driver).move_to_element(tran_button2).click().perform()     # 다음 버튼 클릭
                time.sleep(2)

        except:  # 에러가 나면 다음장을 클릭해라
            # 다음장 클릭
            CSS_tran2="a._65Bje.coreSpriteRightPaginationArrow"             # 다음 버튼 정의
            tran_button2 = driver.find_element_by_css_selector(CSS_tran2)  # 다음 버튼 find
            AC(driver).move_to_element(tran_button2).click().perform()     # 다음 버튼 클릭
            time.sleep(2)

    print(dict)

    # 엑셀(테이블) 형식으로 만들기
    result_df = pd.DataFrame.from_dict(dict, 'index')

    n = result_df['picture'].count()

    # csv 형식(엑셀이랑 비슷한 파일)으로 저장
    result_df.to_csv("data/food({}).csv".format(keyword), encoding='euc-kr')

0  새마을쫄면분식
1  afuri instanoodle ramen l4l f4f fff tflers
3 
4 
5 
6  연어 온더테이블 홈카페 점심 일상 감성 일상스타그램 요리 먹스타그램 음식 사진 요리스타그램 다이어터 식단일기 다이어트 식단 다이어트식단 건강식 주말 daily homecooking foodstyling onthetable plating lunch salomon instafood healthyfood cooking avocado
7  홈쿡 집밥 요리 푸드스타그램 메뉴 샘표로부터소정의적립금을받았습니다 음식 맛스타그램 새댁요리 디저트 비오는날 집스타그램 온더테이블 위드테이블 테이블 인테리어 연두 연두순 연두의집밥 onthetable homecook cooking table koreafood foodstagram foodie
9  홍대개미 덮밥 먹스타그램 맛스타그램 푸드스타그램 먹스타 맛스타 맛점 먹방스타그램 먹방 인스타푸드 음식스타그램 음식 맛집 먹부림 먹스타 먹방그램 냠냠 일상 일상스타그램 푸드 food foodstagram foodpics instafood jmt 옴뇸뇸 존맛 짱맛
10  선물 여친선물 보테가베네타 부츠 보테가 남친선물
11  태안 안면도 안면암 바베큐 펜션 수영장 음식 볼거리 광명 비속을뚫고 숫불매운등갈비 태풍
12  맛스타그램 먹스타그램 선팔하면맞팔 음식 음식스타그램
13  상무초밥
14  맛점
15  선물 여친선물 보테가베네타 부츠 보테가 남친선물
16  맛집스타그램 05년생 음식
17  초당순두부 두부 순두부 두부전골 손두부 부드럽다 녹는다 덜담백 추천 강릉맛집 가볼만한맛집 내부 깔끔 정돈 먹스타그램 맛스타그램 음식 Food 좋반 좋아요반사 좋아요 눌러주세요 좋반가요 선팔환영 선팔맞팔
18  초당옥수수 마켓컬리 간식 푸드 음식 혼점 다이어트 옥수수
19  밥상일기 버섯크림리조또 우니테이블
{0: {'picture': 'https://scontent-ssn1-1.cdninstagram.com/v/t51.2885-15/e

크롤링 할때 번호를 띄어넘는 것은 html element가 변경되었거나 읽히지 못한 경우 이기에 재확인 해서 입력. 

크롤링은 되었는데 해시태그가 되지 않는 경우 # 태그 량이 많아서 + 버튼에 가렸기에 # 를 띄어넘음.

In [9]:
num_pic = len(result_df['picture'])  # 크롤링한 데이터 개수 확인
num_pic

18

In [10]:
# 이미지들 image 폴더에 다운받기
import urllib.request

for i in range(0, 20):
    try:
        index=result_df['picture'][i]
        urllib.request.urlretrieve(index, "image/food{0}_{1}.jpg".format(date, i))        
    except:
        pass